In [2]:
# Load Packages - KEEP MINIMAL FOR FISHTANK COMPATIBILITY
import pandas as pd
import numpy as np
import os
import sys

In [3]:
# set locations for working files
if len(sys.argv) != 3:
    print("Usage: python3 iter-params.py <automation_dir> <attpcroot_dir>")
    print('Assuming testing directories')
    automation_dir = '/mnt/analysis/e17023/Adam/GADGET2/'
    attpcroot_dir = '/mnt/analysis/e17023/Adam/ATTPCROOTv2/'
else:
    # Automation directory
    automation_dir = sys.argv[1]
    
    # ATTPCROOTv2 directory
    attpcroot_dir = sys.argv[2]

Usage: python iter-params.py <automation_dir> <attpcroot_dir>
Assuming testing directories


In [78]:
def indicator_file(file_type, indicator_directory=automation_dir):
    df = pd.DataFrame([0])
    df.to_csv(indicator_directory + file_type + '.csv', index=False)
    print(file_type + ' FILE CREATED')

In [16]:
def energy_to_momentum(energy, particle):
    # input energy in KeV, convert to MeV
    energy = energy/1000

    # Mass values from NIST
    if particle == 'a':
        mass = 3727.3794066 # MeV/c^2
    elif particle == 'p':
        mass = 938.27208816 # MeV/c^2
    else:
        indicator_file('STOP')
        raise Exception('Error: particle must be "a" or "p"')
    momentum = np.sqrt(2*mass*energy)/1000 # GeV/c
    return momentum

energy_to_momentum(506, 'a')

0.06141748903593503

In [80]:
parameters = pd.read_csv(automation_dir + 'simInput/parameters.csv')

In [22]:
# check for and complete any active simulations

# 0 = inactive
# 1 = active
# 2 = complete

# todo: check for changes that require a rebuild

if not parameters['Sim'].is_unique:
    indicator_file('STOP')
    raise Exception('Simulation names are not unique')

active_sims = parameters[parameters['Status'] == 1]
if len(active_sims) > 0:
    
    if len(active_sims) > 1:
        indicator_file('STOP')
        raise Exception('More than one active simulation')
    
    # Search for output.h5 and rename
    Complete = False
    for filename in os.listdir(automation_dir + 'simOutput/'):
        f = os.path.join(automation_dir + 'simOutput/', filename)
        # checking if it is a file
        if os.path.isfile(f):
            if filename == 'output.h5':
                os.rename(f, automation_dir + 'simOutput/hdf5/' + active_sims.loc[active_sims.index[0],'Sim']+'.h5')
                Complete = True
    
    # Set Status in parameters
    if Complete:
        parameters.loc[active_sims.index[0], 'Status'] = 2
        print('Simulation', parameters.loc[active_sims.index[0], 'Sim'] + ' complete')
        
    else:
        indicator_file('STOP')
        raise Exception('Could not find output file')

STOP FILE CREATED


Exception: Could not find output file

In [ ]:
# Determine next simulation to run and mark as active
inactive_sims = parameters[parameters['Status'] == 0]
if len(inactive_sims) == 0:
    indicator_file('STOP')
    parameters.to_csv(automation_dir + 'simInput/parameters.csv', index=False)
    raise Exception('Finished with all simulations')
else:
    if len(active_sims) > 0: # not first iteration
        # check for parameter sets that don't require a rebuild but have not been run
        for i in inactive_sims.index:
            score = 0
            building_params = ['P0', 'E0', 'P1', 'E1']
            for param in building_params:
                score += inactive_sims.loc[i, param] == active_sims.loc[active_sims.index[0], param]
            if score == 4:
                active_sim = i
                break # found a match, break out of loop
            else:
                if i == inactive_sims.index[-1]: # no match found
                    active_sim = inactive_sims.index[0]
                    indicator_file('BUILD')
                    
    else: # first iteration, no active sims to compare to
        active_sim = inactive_sims.index[0]
        indicator_file('BUILD')
    
    parameters.loc[active_sim, 'Status'] = 1
    print("next simulation: ", parameters.loc[active_sim, 'Sim'])

next simulation:  Reference


In [ ]:
# run check on parameters to confirm they are valid
# TODO - generally fixes itself by erroring out, but should be fixed to avoid wasting time on bad parameters

In [34]:
# Modify GADGET.sim.par
with open(automation_dir + 'simInput/templates/GADGET.sim.par', 'r') as file :
    filedata = file.readlines()
    
# replace target parameters
for param in parameters.columns:
    for i, line in enumerate(filedata):
        if param == line.split(':')[0]:
            # Line composition:     param:ptype_t   paramval   # units / comments
            
            ptype = line.split(':')[1].split('_')[0]
            paramval = parameters.loc[active_sim, param]
            filedata[i] = param + ': ' + ptype + '_t     ' + str(paramval) + '     #' + line.split('#')[1]
 
# write file
with open(automation_dir + 'simInput/queue/GADGET.sim.par', 'w') as file:
    file.writelines(filedata)


In [51]:
# Modify AtTPC20MgDecay_pag.cxx (Generators)
P0 = parameters.loc[active_sim, 'P0']; E0 = parameters.loc[active_sim, 'E0']
P1 = parameters.loc[active_sim, 'P1']; E1 = parameters.loc[active_sim, 'E1']

ParticleString = str(P0)
if E1 != 0:
    ParticleString = ParticleString + str(P1)

ParticleString = ParticleString.upper()


# Determine order of particles for generator file and existance of file
# allows for two-particle generators to be called in any order
if os.path.isfile(automation_dir + 'simInput/templates/Generator' + ParticleString + '.txt'):
    pass
elif os.path.isfile(automation_dir + 'simInput/templates/Generator' + ParticleString[::-1] + '.txt'):
    ParticleString = ParticleString[::-1]
    P0 = parameters.loc[active_sim, 'P1']; E0 = parameters.loc[active_sim, 'E1']
    P1 = parameters.loc[active_sim, 'P0']; E1 = parameters.loc[active_sim, 'E0']
else:
    indicator_file('STOP')
    print('Generator file', ParticleString ,'does not exist')
    raise Exception('Generator file not found')

# TEST FOR EXISTING GENERATOR FILE AND EDIT WITH PARTICLE ENERGIES
with open(automation_dir + 'simInput/templates/Generator' + ParticleString + '.txt', 'r') as file :
    filedata = file.readlines()
    
# locate and replace particle energies (Comment lines in generators with P0 E0 or P1 E1 to specify which line to replace)
for i, line in enumerate(filedata):
    if 'P0 E0' in line.split('//')[-1]:
        filedata[i] = line.split('=')[0] + '= ' + str(energy_to_momentum(E0, P0)) + '; // P0 E0\n'

    if 'P1 E1' in line.split('//')[-1]:
        filedata[i] = line.split('=')[0] + '= ' + str(energy_to_momentum(E1, P1)) + '; // P1 E1\n'
    
    if "Seed" in line.split('//')[-1]:
        if 'Seed' in parameters.columns:
            if parameters.loc[active_sim, 'Seed'] != 0:
                filedata[i] = line.split('(')[0] + '(' + str(parameters.loc[active_sim, 'Seed']) + '); // Seed\n'

# write file
with open(automation_dir + 'simInput/queue/AtTPC20MgDecay_pag.cxx', 'w') as file:
    file.writelines(filedata)

PA
   Double32_t pabsProton = 0.04745369333976018; // P0 E0

   Double32_t pabsAlpha = 0.0610522678252004; // P1 E1



In [81]:
# MODIFY Mg20_test_sim.C 
with open(automation_dir + 'simInput/templates/Mg20_test_sim_pag.C', 'r') as file:
    filedata = file.readlines()

# Modify particle momentum
for i, line in enumerate(filedata):
    if 'P0 E0' in line.split('//')[-1]:
        filedata[i] = line.split('(')[0] + '(' + str(E0 / 1000000) + ', 1); // P0 E0\n'

    if 'P1 E1' in line.split('//')[-1]:
        if E1 == 0:
            filedata[i] = '// ' + line
        else:
            filedata[i] = line.split('(')[0] + '(' + str(E1 / 1000000) + ', 1); // P1 E1\n'

# modify particle origin
for i, line in enumerate(filedata):
    if 'bounds' in line.split('//')[-1]:
        Xb = 0; Yb = 0; Zb1 = 10; Zb2 = 40
        if 'Xb' in parameters.columns:
            Xb = parameters.loc[active_sim, 'Xb']
        if 'Yb' in parameters.columns:
            Yb = parameters.loc[active_sim, 'Yb']
        if 'Zb1' in parameters.columns:
            Zb1 = parameters.loc[active_sim, 'Zb1']
        if 'Zb2' in parameters.columns:
            Zb2 = parameters.loc[active_sim, 'Zb2']
            
        filedata[i] = line.split('(')[0] + '(' + str(-Xb) + ', ' + str(-Yb) + ', ' + str(Zb1) + ', ' + str(Xb) + ', ' + str(Yb) + ', ' + str(Zb2) + '); // bounds\n'

# modify number of particles
if 'N' in parameters.columns:
    filedata[0] = filedata[0].split('=')[0] + '= ' + str(int(parameters.loc[active_sim, 'N'])) + ',' + filedata[0].split(',')[-1]

# modify directory for accurate locations
#for i, line in enumerate(filedata):
    # test for output file
#    if 'Output file name' in line.split('//')[-1]:
#        filedata[i] = line.split('"')[0] + '"' + attpcroot_dir + 'macro/Simulation/Charge_Dispersion/data/gadgetsim.root"' + line.split('"')[-1] 
#    elif 'Parameter file name' in line.split('//')[-1]:
#        filedata[i] = line.split('"')[0] + '"' + attpcroot_dir + 'macro/Simulation/Charge_Dispersion/data/gadgetpar.root"' + line.split('"')[-1]
#    elif 'GeoFile location' in line.split('//')[-1]:
#        filedata[i] = line.split('"')[0] + '"' + attpcroot_dir + 'macro/Simulation/Charge_Dispersion/data/geofile_proto_full.root"' + line.split('"')[-1]

# write file
with open(automation_dir + 'simInput/queue/Mg20_test_sim_pag.C', 'w') as file:
    file.writelines(filedata)

In [98]:
# Modify rundigi_sim.C
with open(automation_dir + 'simInput/templates/rundigi_sim_CD.C', 'r') as file:
    filedata = file.readlines()

# modify number of particles
for i, line in enumerate(filedata):
    if ' N\n' == line.split('//')[-1]:
        if 'N' in parameters.columns:
            filedata[i] = line.split(',')[0] + ', ' + str(int(parameters.loc[active_sim, 'N'])) + '); // N\n'

# modify Threshold
for i, line in enumerate(filedata):
    if 'Threshold' in line.split('//')[-1]:
        if 'Threshold' in parameters.columns:
            threshold = parameters.loc[active_sim, 'Threshold']
        else:
            threshold = 20

        filedata[i] = line.split('(')[0] + '(' + str(threshold) + '); // Threshold\n'

# modify Charge Dispersion Adjacent Pads
for i, line in enumerate(filedata):
    if 'Charge Dispersion' in line.split('//')[-1]:
        if 'CD' in parameters.columns:
            filedata[i] = line.split('(')[0] + '(' + str(int(parameters.loc[active_sim, 'CD'])) + '); // Charge Dispersion\n'

# Modify directory for accurate locations
#for i, line in enumerate(filedata):
#    if 'gadgetsim root file' in line.split('//')[-1]:
#        filedata[i] = line.split('"')[0] + '"' + attpcroot_dir + 'macro/Simulation/Charge_Dispersion/data/gadgetsim.root"' + line.split('"')[-1]

# write file
with open(automation_dir + 'simInput/queue/rundigi_sim_CD.C', 'w') as file:
    file.writelines(filedata)

In [99]:
# POSSIBLE FUTURE FILE MODIFICATIONS (NOT CURRENTLY IMPLEMENTED)
# Modify R2HMain.cc
# TODO
# Modify media.geo
# TODO
# Modify GADGET_II.C
# TODO
# Modify GADGET_II_lp.C
# TODO

In [ ]:
# Update parameters.csv
parameters.to_csv(automation_dir + 'simInput/parameters.csv', index=False)